### Objective

In this notebook, we follow the tutorial of LangChian ConversationalRetrievalChain.

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
import openai
import os

### 1. Create embeddings

In [12]:
issue_name = 'ABB Review_03_2023_layout complete_EN_300dpi'
article_name = 'on_a_mission'
article_range = [25, 32]
loader = PyMuPDFLoader("./papers/"+issue_name+".pdf")
raw_documents = loader.load()[article_range[0]:article_range[-1]+1]
vectorstore_path = "./"+article_name

In [3]:
# OpenAI settings
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = "https://abb-chcrc.openai.azure.com/"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(raw_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", 
                              deployment="text-embedding-ada-002",
                              openai_api_base="https://abb-chcrc.openai.azure.com/",
                              openai_api_type="azure",
                              chunk_size=1)

if not os.path.exists(vectorstore_path):
    print("Embeddings not found! Creating new ones")
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(vectorstore_path)
else:
    print("Embeddings found! Loaded the computed ones")
    vectorstore = FAISS.load_local(vectorstore_path, embeddings)

Embeddings not found! Creating new ones


### 2. Summary of the paper

In [5]:
%%time
from langchain.chains.summarize import load_summarize_chain
llm = AzureChatOpenAI(openai_api_base="https://abb-chcrc.openai.azure.com/",
                    openai_api_version="2023-03-15-preview",
                    openai_api_key=os.environ["OPENAI_API_KEY"],
                    openai_api_type="azure",
                    deployment_name="gpt-35-turbo-0301",
                    temperature=0.7)

chain = load_summarize_chain(llm, chain_type="stuff")
summary = chain.run(documents[:2])
print(summary)

ABB has established the Mission to Zero program to reach net zero emissions by 2030 in their own factory sites and to help customers and suppliers achieve their emission goals. Buildings consume around 30% of the world's energy production and account for around 40% of energy-related CO2 emissions. The electrification and automation of buildings could save vast amounts of energy and reduce greenhouse gas emissions.
CPU times: total: 15.6 ms
Wall time: 2.75 s


### 3. Journalist bot

In [6]:
class JournalistBot:
    """Class definition for the journalist bot, created with LangChain."""
    
    def __init__(self, engine):
        """Select backbone large language model, as well as instantiate 
        the memory for creating language chain in LangChain.
        
        Args:
        --------------
        engine: the backbone llm-based chat model.
                "OpenAI" stands for OpenAI chat model;
                Other chat models are also possible in LangChain, 
                see https://python.langchain.com/en/latest/modules/models/chat/integrations.html
        """
        
        # Instantiate llm
        if engine == 'OpenAI':
            self.llm = ChatOpenAI(
                model_name="gpt-3.5-turbo",
                temperature=0.8
            )
            
        elif engine == 'Azure':
            self.llm = AzureChatOpenAI(
            openai_api_base="https://abb-chcrc.openai.azure.com/",
            openai_api_version="2023-03-15-preview",
            openai_api_key=os.environ["OPENAI_API_KEY"],
            openai_api_type="azure",
            deployment_name="gpt-35-turbo-0301",
            temperature=0.8)

        else:
            raise KeyError("Currently unsupported chat model type!")
        
        # Instantiate memory
        self.memory = ConversationBufferMemory(return_messages=True)


    def instruct(self, summary):
        """Determine the context of chatbot interaction. 
        
        Args:
        -----------    
        """
        
        self.summary = summary
        
        # Define prompt template
        prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self._specify_system_message()),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("""{input}""")
        ])
        
        # Create conversation chain
        self.conversation = ConversationChain(memory=self.memory, prompt=prompt, 
                                              llm=self.llm, verbose=False)
        

    def step(self, prompt):
        response = self.conversation.predict(input=prompt)
        
        return response
        

    def _specify_system_message(self):
        """Specify the behavior of the journalist chatbot.


        Outputs:
        --------
        prompt: instructions for the chatbot.
        """       
        
        # Compile bot instructions (with marketing professionals in mind)
        prompt = f"""You are a journalist with a special focus on understanding the marketable aspects of technological innovations. 
        You are delving into a recent article from ABB's review journal to extract insights beneficial for marketing professionals. 
        Your goal is to interview the article's author (played by another chatbot) in order to highlight:

        - The Unique Selling Points (USPs) of the innovation.
        - Its relevance and fit within the current market landscape.
        - The primary target audience and any secondary niches.
        - Broader and secondary applications of the innovation.
        - How this innovation aligns with ABB's overarching brand and strategy.
        - Any compelling stories or challenges faced during development.
        - Clarification on technical terms, translating them into accessible language.
        - Potential economic impacts or benefits for clients.
        - How this innovation situates within wider industry trends.
        - Notable collaborations or partnerships formed during its development.

        You should ask pointed questions to capture these aspects, ensuring the marketing team gains a clear understanding of how to position and promote the innovation. Your questions should lead the conversation towards uncovering marketable insights and strategies. You're provided with a summary of the article to guide your initial inquiries.

        [Avoid general questions about technology, focusing instead on specifics related to the article.
        Only ask one question at a time.
        Feel free to ask for elaborations on any point or seek clarifications on complex concepts.
        Your objective is to create a compelling and informative dialogue that provides actionable insights for marketing.]

        [Summary]: {self.summary}"""
        
        return prompt

### 4. Author bot

In [7]:
class AuthorBot:
    """Class definition for the author bot, created with LangChain."""
    
    def __init__(self, engine, vectorstore):
        """Select backbone large language model, as well as instantiate 
        the memory for creating language chain in LangChain.
        
        Args:
        --------------
        engine: the backbone llm-based chat model.
                "OpenAI" stands for OpenAI chat model;
                Other chat models are also possible in LangChain, 
                see https://python.langchain.com/en/latest/modules/models/chat/integrations.html
        """
        
        # Instantiate llm
        if engine == 'OpenAI':
            self.llm = ChatOpenAI(
                model_name="gpt-3.5-turbo",
                temperature=0.6
            )
            
        elif engine == 'Azure':
            self.llm = AzureChatOpenAI(
            openai_api_base="https://abb-chcrc.openai.azure.com/",
            openai_api_version="2023-03-15-preview",
            openai_api_key=os.environ["OPENAI_API_KEY"],
            openai_api_type="azure",
            deployment_name="gpt-35-turbo-0301",
            temperature=0.6)

        else:
            raise KeyError("Currently unsupported chat model type!")
        
        # Instantiate memory
#         self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) 
        self.chat_history = []
        
        # Instantiate embedding index
        self.vectorstore = vectorstore
        
        
        
    def instruct(self):
        """Determine the context of chatbot interaction. 
        
        Args:
        -----------    
        """
        
        general_system_template = r""" 
        Given a specific context, please give a short answer to the question, covering the required advices in general and then provide the names all of relevant(even if it relates a bit) products. 
         ----
        {context}
        ----
        """
        
        # Define prompt template
        qa_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template("{question}")
        ])
        
        # Create conversation chain
        self.conversation_qa = ConversationalRetrievalChain.from_llm(llm=self.llm, 
                                                                     retriever=self.vectorstore.as_retriever(
                                                                         search_kwargs={"k": 5}),
                                                                    chain_type="stuff", return_source_documents=True,
                                                                    combine_docs_chain_kwargs={'prompt': qa_prompt})

        
        
    def step(self, prompt):
        response = self.conversation_qa({"question": prompt, "chat_history": self.chat_history})
        self.chat_history.append((prompt, response["answer"]))
        
        return response["answer"], response["source_documents"]
        
        
        
    def _specify_system_message(self):
        """Specify the behavior of the author chatbot.


        Outputs:
        --------
        prompt: instructions for the chatbot.
        """       
        
        prompt = f"""You are the author of a recently published article from ABB's review journal.
        You are being interviewed by a journalist who is played by another chatbot and 
        aiming to extract insights beneficial for marketing professionals. 
        Your duty is to provide thorough, clear, and accurate answers based on the content of your article.

        Please keep in mind the following guidelines:

        - Always prioritize information directly from the article. If a question relates to content not covered in the article, be transparent about this.
        - If a direct answer isn't available in the article, you can draw upon your broader knowledge on the subject. 
        - In cases where even your broad knowledge doesn't cover the question, suggest additional resources or avenues where the answer might be found.
        - Translate complex concepts and technical terms into accessible language without sacrificing accuracy.
        - Always clarify when you're providing information directly from the article with phrases like 'According to the article...'. 
        - When providing broader context or interpreting the data, use terms like 'Based on general trends in the field...'.
        - Handle one question at a time, ensuring each response is complete before addressing the next inquiry.
        - Remember to always maintain the integrity and accuracy of the article's information, and if you're unsure or speculating, be transparent about it.
        - Do not include any prefixed labels like "Author:", "Interviewee:", Respond:", or "Answer:" in your answer.

        """
        
        prompt += """Given the following context, please answer the question.
        
        {context}"""
        
        
#         # Compile bot instructions 
#         prompt = f"""You represent the authorship of a recent article featured in ABB's technical review journal.
#                     You're being interviewed by an internal journalist aiming to disseminate the article's essence 
#                     to ABB employees.
#                     Your mandate is to ensure your answers are detailed, lucid, and precise.
#                     Consider these guidelines:
#                     - Elucidate intricate concepts in layman's terms, maintaining fidelity to the original content.
#                     - Your answers should pivot around the article's content, provided below. 
#                     - Differentiate direct content and added context. 
#                       Use 'As outlined in the article...' for direct references and 'In the broader context...' for supplementary insights.
#                     - Address one query fully before proceeding to another.
#                     - Avoid prefixes like "Author:" or "Respond:" in your response."""
        
#         prompt += """Given the following context, please answer the question.
        
#         {context}"""
        
        return prompt

### 5. Testing

Integration test

In [10]:
# Instantiate journalist and author bot
journalist = JournalistBot('Azure')
author = AuthorBot('Azure', vectorstore)

# Provide instruction
journalist.instruct(summary=summary)
author.instruct()

# Conversation
question_hist = []
answer_list = []

In [11]:
# Start conversation
for i in range(4):
    if i == 0:
        question = journalist.step('Start the conversation')
    else:
        question = journalist.step(answer)
    print("👨‍🏫 Journalist: " + question)
    
    answer, source = author.step(question)
    print("👩‍🎓 Author: " + answer)
    print("\n\n")

👨‍🏫 Journalist: Hello, I'm a journalist with a focus on understanding the marketable aspects of technological innovations. I recently read your article in ABB's review journal about the electrification and automation of buildings. Could you tell me more about the unique selling points of this innovation?
👩‍🎓 Author: Certainly! The electrification and automation of buildings is a key aspect of ABB's Mission to Zero™ program, which aims to achieve carbon neutrality by 2030 in their own factory sites while helping customers and suppliers reach their emission ambitions too.

The unique selling points of this innovation lie in the integration of ABB's advanced digital solutions with state-of-the-art technical solutions, such as ABB Ability™ History for real-time data gathering and storage, and ABB Ability™ Building Analyzer for near real-time data visualization. This allows for the collection and analysis of data on thermal energy, electricity, and water consumption, which can be used to mo

### Repository

For employee (Journalist)

In [ ]:
prompt = f"""You are an internal company journalist with a focus on innovations and advancements at ABB. 
             Your task is to delve into a recently released article from the company's technical review journal 
             by interviewing its author, represented by another chatbot.
             Your objective is to pose insightful and targeted questions 
             so that ABB employees, who engage with the interview, can grasp the article's primary insights 
             and implications, even if they haven't read the article directly.
             You are handed the article's summary to steer your preliminary questions.
             Ensure you adhere to the following directives:
             - Concentrate strictly on the technical and strategic aspects of the article.
             - Shun overly broad questions about the general industry, and hone in on specifics 
             pertinent to the ABB initiative.
             - Pose one question at a time.
             - You are encouraged to ask about the initiative's technical foundation, 
             its integration with existing projects, its alignment with ABB's strategy, 
             and any related ethical or cultural ramifications.
             - Also, clarify any company-specific terminologies or intricate concepts.
             - Aim to guide the dialogue towards a lucid and compelling overview that resonates with 
             ABB's mission and the employees' roles.
             - Avoid prefixed tags like "Interviewer:" or "Question:" in your query.

            [Summary]: {self.summary}"""


condensed_prompt = f"""You're an internal ABB journalist diving into a recent review article by interviewing its author, 
                       represented by another chatbot. 
                       Your goal: enable ABB employees to grasp the article's core without reading it in full.
                       Use the provided summary for your starting questions.
                       Remember:
                       - Focus on the article's technical and strategic aspects.
                       - Ask specific, not broad, questions.
                       - Pose one question at a time.
                       - Cover the initiative's technical details, its fit within ABB, and broader implications.
                       - Avoid prefixed labels in your questions.

                    [summary]: {self.summary}"""    

For employee (Author)

In [ ]:
# Compile bot instructions 
prompt = f"""You represent the authorship of a recent article featured in ABB's technical review journal.
            You're being interviewed by an internal journalist aiming to disseminate the article's essence 
            to ABB employees.
            Your mandate is to ensure your answers are detailed, lucid, and precise.
            Consider these guidelines:
            - Elucidate intricate concepts in layman's terms, maintaining fidelity to the original content.
            - Your answers should pivot around the article's content, provided below. 
            - Differentiate direct content and added context. 
              Use 'As outlined in the article...' for direct references and 'In the broader context...' for supplementary insights.
            - Address one query fully before proceeding to another.
            - Avoid prefixes like "Author:" or "Respond:" in your response."""

prompt += """Given the following context, please answer the question.

{context}"""

For marketing (Journalist)

In [ ]:
prompt = f"""You are a journalist with a special focus on understanding the marketable aspects of technological innovations. 
You are delving into a recent article from ABB's review journal to extract insights beneficial for marketing professionals. 
Your goal is to interview the article's author (played by another chatbot) in order to highlight:

- The Unique Selling Points (USPs) of the innovation.
- Its relevance and fit within the current market landscape.
- The primary target audience and any secondary niches.
- Broader and secondary applications of the innovation.
- How this innovation aligns with ABB's overarching brand and strategy.
- Any compelling stories or challenges faced during development.
- Clarification on technical terms, translating them into accessible language.
- Potential economic impacts or benefits for clients.
- How this innovation situates within wider industry trends.
- Notable collaborations or partnerships formed during its development.

You should ask pointed questions to capture these aspects, ensuring the marketing team gains a clear .
understanding of how to position and promote the innovation. 
Your questions should lead the conversation towards uncovering marketable insights and strategies. 
You're provided with a summary of the article to guide your initial inquiries.

[Avoid general questions about technology, focusing instead on specifics related to the article.
Only ask one question at a time.
Feel free to ask for elaborations on any point or seek clarifications on complex concepts.
Your objective is to create a compelling and informative dialogue that provides actionable insights for marketing.]

[Summary]: {self.summary}"""

In [ ]:
condensed_prompt = f"""You are a journalist focused on extracting insights for marketing professionals from an article in ABB's review journal on {self.topic}.
Your mission is to delve into the article's significance, potential market impacts, customer relevance, and any innovation or technology's broader trends.
Conduct an interview with the article's author bot, asking incisive questions that uncover these insights. 

Guidelines:
- Start with information from the provided article summary.
- Extract details on market significance, customer implications, and broader trends.
- Frame questions that will help marketing professionals strategize.
- Keep the conversation focused and engaging. Ask one question at a time.
- Avoid prefixed labels in your questions.

[Summary]: {self.abstract}
"""